# Семинар 4: Текстовые признаки и подбор гиперпараметров

Inspired by: https://github.com/esokolov/ml-course-hse/blob/master/2021-fall/seminars/sem04-features.ipynb

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Murcha1990/ML_math_2022/blob/main/Семинары/sem04-text_data.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import IPython.display as ipd

In [ ]:
!wget https://raw.githubusercontent.com/Murcha1990/ML_math_2022/main/Семинары/data/anime_clean.csv

anime_data = pd.read_csv('anime_clean.csv')

In [ ]:
import ast

anime_data["genre"] = anime_data.genre.apply(ast.literal_eval)

In [ ]:
def render(frame, exclude=["link"], max_rows=10):
    ipd.display(ipd.HTML(frame.head(max_rows).drop(columns=exclude).to_html(escape=False, formatters={"img_url": lambda url: f'<img src="{url}"/>'})))


render(anime_data)

In [ ]:
texts = anime_data.synopsis.fillna("").to_list()

### Bag-of-words

Самый очевидный способ формирования признакового описания текстов — векторизация. Пусть у нас имеется коллекция текстов $D = \{d_i\}_{i=1}^l$ и словарь всех слов, встречающихся в выборке $V = \{v_j\}_{j=1}^d.$ В этом случае некоторый текст $d_i$ описывается вектором $(x_{ij})_{j=1}^d,$ где
$$x_{ij} = \sum_{v \in d_i} [v = v_j].$$

Таким образом, текст $d_i$ описывается вектором количества вхождений каждого слова из словаря в данный текст.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(encoding='utf8', min_df=1)
vectorizer.fit_transform(texts)

### TF-IDF

Ещё один способ работы с текстовыми данными — [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (**T**erm **F**requency–**I**nverse **D**ocument **F**requency). Рассмотрим коллекцию текстов $D$.  Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте:
$$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$
где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
1. Inverse Document Frequency
$$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$
где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину:

$$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D)$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(encoding='utf8', min_df=5, stop_words="english")
vectorizer.fit_transform(texts)

Заметим, что одно и то же слово может встречаться в различных формах (например, "сотрудник" и "сотрудника"), но описанные выше методы интерпретируют их как различные слова, что делает признаковое описание избыточным. Устранить эту проблему можно при помощи **лемматизации** и **стемминга**.

### Стемминг

[**Stemming**](https://en.wikipedia.org/wiki/Stemming) –  это процесс нахождения основы слова. В результате применения данной процедуры однокоренные слова, как правило, преобразуются к одинаковому виду.

**Примеры стемминга:**

| Word        | Stem           |
| ----------- |:-------------:|
| вагон | вагон |
| вагона | вагон |
| вагоне | вагон |
| вагонов | вагон |
| вагоном | вагон |
| вагоны | вагон |
| важная | важн |
| важнее | важн |
| важнейшие | важн |
| важнейшими | важн |
| важничал | важнича |
| важно | важн |

[Snowball](http://snowball.tartarus.org/) – фрэймворк для написания алгоритмов стемминга. Алгоритмы стемминга отличаются для разных языков и используют знания о конкретном языке – списки окончаний для разных чистей речи, разных склонений и т.д. Пример алгоритма для русского языка – [Russian stemming](http://snowballstem.org/algorithms/russian/stemmer.html).

In [ ]:
import nltk

In [ ]:
stemmer = nltk.stem.snowball.EnglishStemmer()

In [ ]:
stemmer.stem("have"), stemmer.stem("having"), stemmer.stem("had")

In [ ]:
def stem_text(text, stemmer):
    return ' '.join([stemmer.stem(word) for word in text.split()])


stemmed_texts = [stem_text(text, stemmer) for text in texts]
vectorizer.fit_transform(stemmed_texts)


In [ ]:
nltk.download('omw-1.4')

lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
stemmer.stem("corpora"), lemmatizer.lemmatize("corpora")

In [ ]:
def lemmatize_text(text, lemmatizemer):
    return ' '.join([lemmatizemer.lemmatize(word) for word in text.split()])


lemmatized_texts = [lemmatize_text(text, lemmatizer) for text in texts]
vectorizer.fit_transform(lemmatized_texts)


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, r2_score

metrics = {
    "EV": explained_variance_score,
    "MAE": mean_absolute_error,
    "MSE": mean_squared_error,
    "R2": r2_score
}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texts, anime_data.score, test_size=0.2, random_state=42)

In [ ]:
pipeline = Pipeline((
    ("vectorizer", TfidfVectorizer(min_df=5, stop_words="english")),
    ("model", LinearRegression())
))

In [ ]:
pipeline.fit(X_train, y_train)

for metric_name, metric in metrics.items():
    print(f"{metric_name}: {metric(y_test, pipeline.predict(X_test))}")

In [ ]:
pipeline.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
param_grid = [
    {
        "vectorizer__max_features": [1000, 5000, 10000],
        "model": [LinearRegression()],
    },
    {
        "vectorizer__max_features": [1000, 5000, 10000],
        "model": [Ridge(), Lasso()],
        "model__alpha": [0.1, 1, 10],
    },
]

In [ ]:
searcher = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1, scoring="r2")
searcher.fit(X_train, y_train)

In [ ]:
pd.DataFrame(searcher.cv_results_)

In [ ]:
for metric_name, metric in metrics.items():
    print(f"{metric_name}: {metric(y_test, searcher.predict(X_test))}")

### Как можем улучшить?

- Использовать более продвинутые модели (например [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html))
- Добавить категориальные и численные признаки
- Расширить признаковое описание
- Попробовать другие модели